In [1]:
%load_ext edx_jupyter

In [2]:
x = %vertica SELECT 'Connected to Vertica'
print(x.loc[0][0])

Connected to Vertica


In [3]:
import datetime

In [4]:
min_date = datetime.date(2016,5,31)

In [8]:
%%vertica (min_date=min_date)
WITH enrolls AS (
    SELECT
        COUNT(*) AS cnt_enrollments
    FROM
        production.d_user_course uc
    JOIN
        business_intelligence.course_master cm ON
            uc.course_id = cm.course_id
            AND course_verification_end_date IS NOT NULL
            AND (
                (pacing_type = 'self_paced' AND course_start_date < NOW())
                OR course_verification_end_date < NOW()
            )
    WHERE
        first_enrollment_time > :min_date
), bookings AS (
    SELECT
        SUM(transaction_amount_per_item) AS sum_bookings
    FROM
        finance.f_orderitem_transactions
    WHERE
        DATE(transaction_date) > :min_date
)
SELECT
    e.cnt_enrollments,
    b.sum_bookings,
    b.sum_bookings / e.cnt_enrollments AS bookings_per_enrollment
FROM bookings b, enrolls e

,cnt_enrollments,sum_bookings,bookings_per_enrollment
0,13192229,22044614.98,1.67103034521307960922


# Our bookings per enrollment in the past year is $1.67

In [7]:
1.67 * 1.30

2.171